In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
class BookSelection:
    if __name__ == "__main__":
        def __init__(self, url, search_keywords):
            self.url = url
            self.search_keywords = search_keywords

        def set_drivers(self):
            try:
                service = Service(executable_path=ChromeDriverManager().install())
                options = Options()
                # 참고 = https://codingstep.tistory.com/entry/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%85%80%EB%A0%88%EB%8B%88%EC%9B%80-%ED%81%AC%EB%A1%AC-%EC%98%B5%EC%85%98-python-selenium-chrome-optionsFeat-headless-window-size-user-agent-etc
                #지정한 user-agent로 설정합니다.
#                 user_agent = "Mozilla/5.0 (Linux; Android 9; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.83 Mobile Safari/537.36"
#                 options.add_argument('user-agent=' + user_agent)
#                 options.add_argument('headless') #headless모드 브라우저가 뜨지 않고 실행됩니다.
#                 options.add_argument('--blink-settings=imagesEnabled=false') #브라우저에서 이미지 로딩을 하지 않습니다.
#                 options.add_argument('--mute-audio') #브라우저에 음소거 옵션을 적용합니다.
#                 options.add_argument('--window-size= x, y') #실행되는 브라우저 크기를 지정할 수 있습니다.
#                 options.add_argument('--start-maximized') #브라우저가 최대화된 상태로 실행됩니다.
#                 options.add_argument('--start-fullscreen') #브라우저가 풀스크린 모드(F11)로 실행됩니다.
#                 options.add_argument('incognito') #시크릿 모드의 브라우저가 실행됩니다.
                driver = webdriver.Chrome(service=service, options=options)  ##버전에 상관없이 현재 OS에 설치된 크롬브라우저 사용
                return driver
            except Exception as e:
                print(e)
        
        def get_url(self, driver):
            try:
                driver.get(self.url)
                driver.implicitly_wait(time_to_wait=10)
            except Exception as e:
                print(e)
                driver.quit()
                
        def get_searchKeywords(self, driver):
            try:
                search_input = driver.find_element(By.XPATH, "/html/body/div[4]/div[1]/div[1]/div[1]/form[2]/div/div/input")
                search_input.send_keys(self.search_keywords)
                driver.find_element(By.XPATH, "/html/body/div[4]/div[1]/div[1]/div[1]/form[2]/div/input").click()
                driver.implicitly_wait(time_to_wait=10)
            except Exception as e:
                print(e)
                driver.quit()
        
        def get_currentPage(self, driver):
            try:
                currentPage = driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[9]/div[1]/ul/li/strong").text
                return currentPage
            except Exception as e:
                print(e)
                driver.quit()
                
        def get_totalPage(self, driver):
            try:
                totPageCnt = driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[9]/div[1]/span/span").text
                return totPageCnt
            except Exception as e:
                print(e)
                driver.quit()
        
        def get_searchList(self, driver):
            try:
                searchList = driver.find_element(By.ID, "search_list")
#                 searchList = driver.find_element(By.ID, "search_gallery")
                return searchList
            except Exception as e:
                print(e)
                driver.quit()
                
        def click_moreButton(self, driver):
            try:
                if len(driver.find_elements(By.XPATH, ".//a[@class='btn_small btn_more']")):
                    for bsbm in driver.find_elements(By.XPATH, ".//a[@class='btn_small btn_more']"):
                        if bsbm.text == "더보기":
                            bsbm.click()
                            time.sleep(2)
            except Exception as e:
                print(e)

                
        def crawling_detailPage(self, driver, searchList):
            try:
                # searchListLength = len(searchList.find_elements(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[8]/form[1]/table/tbody/tr/td[2]/div[2]/a")) ## 펼쳐보기에서 title Link가져오기
                searchListLength = len(searchList.find_elements(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[8]/form[2]/table/tbody/tr/td/div/div[4]/a")) ## 간략보기에서 title Link가져오기
                list_count = 0

                # while (list_count < result_length): change loop -> while loop to for loop
                for list_count in range(searchListLength): 
                    print("Crawling number = "+str(list_count+1)+"/"+str(searchListLength))
                #     titleLinks = driver.find_elements(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[8]/form[1]/table/tbody/tr/td[2]/div[2]/a") ## 펼쳐보기
                    titleLinks = driver.find_elements(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[8]/form[2]/table/tbody/tr/td/div/div[4]/a") ## 간략보기
                    titleLinks[list_count].click()
                    time.sleep(5)

                    detailInfo = dict()
                    
                    # class="box_detail_point"
                    detailPoints = driver.find_elements(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[2]/form/div[1]/*")
                    for detail in detailPoints:
                        detailInfo[detail.get_attribute("class")] = detail.text

                    # class="book_keyword"
                    keywords = []
                    if len(driver.find_elements(By.CLASS_NAME, "book_keyword")) > 0:
                        for keyword in driver.find_elements(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[2]/form/div[2]/div[4]/div[2]/*"):
                            keywords.append(keyword.text)
                        detailInfo['book_keyword'] = ", ".join(keywords)

                    detailcontentList = []
                    contents = driver.find_elements(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[5]/div[1]/div[3]/*")

                    # more button click
                    for content in contents:
                        self.click_moreButton(content)

                    # "content"
                    for content in contents:
                        detailcontentList.append(content.text)
                    detailInfo['content'] = "\n".join(detailcontentList)
                    # "URL"
                    detailInfo['url'] = driver.current_url
                    print(detailInfo)
                    driver.back()
                #     list_count+=1
                    time.sleep(2)
            
                return detailInfo
            except Exception as e:
                print(e)

    else:
        exit()

In [3]:
bookSelection = BookSelection("http://www.kyobobook.co.kr/", "python")
driver = bookSelection.set_drivers()
bookSelection.get_url(driver)
bookSelection.get_searchKeywords(driver)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/administrator/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


In [4]:
searchTabList = driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[1]/ul").text.split('\n')
subTabList = driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[2]/ul").text.split('\n')
categoryList = driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[3]/table/tbody/tr[1]/td/ul").text.split('\n')

In [5]:
print(searchTabList)
print(subTabList)
print(categoryList)

['상품(5,299)', '중고장터(532)', '스토리K(25)', '리뷰(1,262)', '책소개.목차(3,400)']
['전체 5,299', '국내도서 1,157', '외국도서 3,605', 'eBook 500', 'sam 90', '음반/영상 37', '기프트 0']
['컴퓨터/IT 1,111', '어린이(초등) 11', '경제/경영 11', '과학 7', '청소년 3', '정치/사회 3', '취업/수험서 2', '자기계발 2']


In [6]:
## 리스트 보기 개수 조절
driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[6]/div[2]/span[2]").click()
time.sleep(1)

driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[6]/div[2]/span[2]/ul/li[3]").click()
time.sleep(2)

## 리스트 간략히 보기
driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[6]/div[2]/span[3]").click()
time.sleep(1)

driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[6]/div[2]/span[3]/ul/li[2]/a").click()
time.sleep(2)

totalPages = bookSelection.get_totalPage(driver)
print("totPageCnt = "+str(totalPages))

totPageCnt = 89


## 페이지 처리, 상세페이지 크롤링

In [7]:
for page in range(int(totalPages)):
    currentPage = bookSelection.get_currentPage(driver)
    print("CurrentPage = "+str(currentPage))
    
    ## Crawling List    
    searchList = bookSelection.get_searchList(driver)
    ## 상세페이지 Crawling
    detailInfo = bookSelection.crawling_detailPage(driver, searchList)
    
    if currentPage != totalPages:
        driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[9]/div[1]/a[3]").click()
    else:
        print("Complete crawling of all pages!!")
        break
    time.sleep(2)

CurrentPage = 1
Crawling number = 1/60
{'icon': '', 'title': '혼자 공부하는 파이썬 파이썬 최신 버전 반영', 'author': '윤인성 지음 | 한빛미디어 | 2019년 06월 10일 출간', 'review': 'Klover\n총 4 중\n4 9.6 (리뷰 218개)\n클로버 리뷰쓰기', 'rank': '국내도서 주간베스트 293위 | 컴퓨터/IT 주간베스트 9위 교보문고 베스트셀러', 'book_keyword': '', 'content': '도서·상품정보 1:1 문의하기\nISBN 9791162241882(1162241888)\n쪽수 460쪽\n크기 188 * 258 * 28 mm /1001g 판형알림\n책소개\n이 책이 속한 분야\n컴퓨터/IT > 프로그래밍 언어 > Python(파이썬)\n컴퓨터/IT > 대학교재\n대학교재 > 컴퓨터\n이 책의 주제어\n#프로그래밍 #자습서 #입문자 #손코딩 #전공자\n\n혼자 해도 충분하다! 1:1 과외하듯 배우는 파이썬 프로그래밍 자습서(파이썬 최신 버전 반영)\n27명의 베타리더 검증으로, ‘함께 만든’ 입문자 맞춤형 도서\n이 책은 독학으로 프로그래밍 언어를 처음 배우려는 입문자가, 혹은 파이썬을 배우려는 입문자가 ‘꼭 필요한 내용을 제대로’ 학습할 수 있도록 구성했다. ‘무엇을’, ‘어떻게’ 학습해야 할지조차 모르는 입문자의 막연한 마음을 살펴, 과외 선생님이 알려주듯 친절하게, 그러나 핵심적인 내용만 콕콕 집어준다. 책의 첫 페이지를 펼쳐서 마지막 페이지를 덮을 때까지, 혼자서도 충분히 파이썬을 배울 수 있다는 자신감과 확신이 계속될 것이다!\n\n27명의 베타리더와 함께 구성하여 입문자에게 맞는 난이도, 분량, 학습 요소 등을 적극 반영했다. 어려운 용어와 개념은 한번 더 풀어 쓰고, 복잡한 설명은 눈에 잘 들어오는 그림으로 풀어 냈다. ‘혼자 공부해본’ 여러 입문자의 초심과 눈높이가 책 곳곳에 반영된 것이 이 책의 가장 큰 장점이다.\n상세이미지\n\n\n\

Crawling number = 3/60
Message: disconnected: Unable to receive message from renderer
  (Session info: chrome=97.0.4692.99)
Stacktrace:
0   chromedriver                        0x000000010483ee69 chromedriver + 5160553
1   chromedriver                        0x00000001047c9593 chromedriver + 4679059
2   chromedriver                        0x000000010437d2c8 chromedriver + 172744
3   chromedriver                        0x0000000104368651 chromedriver + 87633
4   chromedriver                        0x00000001043676dc chromedriver + 83676
5   chromedriver                        0x0000000104367adc chromedriver + 84700
6   chromedriver                        0x0000000104386cc4 chromedriver + 212164
7   chromedriver                        0x000000010437faf8 chromedriver + 183032
8   chromedriver                        0x000000010437f7e2 chromedriver + 182242
9   chromedriver                        0x000000010437ff0d chromedriver + 184077
10  chromedriver                        0x0000000104380

Crawling number = 4/60
{'icon': '', 'title': '프로그래머의 뇌 훌륭한 프로그래머가 알아야 할 인지과학의 모든 것', 'author': '펠리너 헤르만스 지음 | 차건회 옮김 | 제이펍 | 2022년 01월 12일 출간', 'review': '총 4 중\n4 10.0 (리뷰 1개)\n클로버 리뷰쓰기', 'rank': '국내도서 주간베스트 241위 | 컴퓨터/IT 주간베스트 7위', 'book_keyword': '', 'content': '도서·상품정보 1:1 문의하기\nISBN 9791191600650(1191600653)\n쪽수 272쪽\n크기 189 * 246 * 18 mm /531g 판형알림\n책소개\n이 책이 속한 분야\n컴퓨터/IT > 개발방법론 > 프로젝트관리\n컴퓨터/IT > 프로그래밍 언어 > 프로그래밍일반\n\n인지과학을 활용한 개발자의 일머리 개선법\n이 책은 인지과학에 기반을 둔 각종 방법론으로 개발자가 새로운 언어나 프레임워크를 빠르게 배워 생산성을 향상하도록 돕는다. 코드를 더 잘 이해하는 것에서 나아가, 버그 방지, 명확한 작명법, 리팩터링, 대규모 코드베이스 개선에 이르기까지 과학적으로 입증된 기법을 제시한다.\n상세이미지\n\n저자소개\n저자 : 펠리너 헤르만스\n작가 정보 관심작가 등록\n프로그래머\nFelienne Hermans 네덜란드 레이던 대학교(Universiteit Leiden)의 부교수로 프로그래밍 교육과 프로그래밍 언어를 연구한다. 암스테르담 자유 대학교(Vrije Univ... 더보기\n\n\n역자 : 차건회\n작가 정보 관심작가 등록\n프로그래머 번역가/통역사＞영어\n현재 미국 로스앤젤레스의 Shell Recharge Solutions에서 자바 백엔드 테크 리드로 일하고 있다. 자바, 스프링 부트, 메시지 큐, 일래스틱서치, 레디스 등 자바 백엔드의 전반... 더보기\n\n목차\n옮긴이 머리말 xi\n추천의 글 xiii\n베타리더 후기 xvi\n추천서문(존 스키트) xviii\n

Crawling number = 5/60
{'icon': '', 'title': '비전공자를 위한 이해할 수 있는 IT 지식', 'author': '최원영 지음 | 티더블유아이지 | 2020년 07월 14일 출간', 'review': 'Klover\n총 4 중\n4 8.5 (리뷰 148개)\n클로버 리뷰쓰기', 'rank': '국내도서 주간베스트 219위 | 컴퓨터/IT 주간베스트 6위 교보문고 베스트셀러', 'book_keyword': '', 'content': '도서·상품정보 1:1 문의하기\nISBN 9791196918033(1196918031)\n쪽수 240쪽\n크기 172 * 234 * 14 mm /535g 판형알림\n책소개\n이 책이 속한 분야\n컴퓨터/IT > IT일반 > IT일반서\n이 책의 주제어\n#IT기본문법 #프로그래밍언어 #운영체제 #네트워크 #API #JSON #데이터베이스 #이미지처리 #프레임워크 #라이브러리\n\nIT 시대의 필수 교양서\n우리는 매일 스마트폰으로 메시지를 보내고, 컴퓨터로 일을 합니다. 또 음식점에서 키오스크로 주문하고, 가상현실에서 게임을 즐깁니다. IT는 더 이상 전문가들만의 이야기가 아닙니다. 우리의 삶 곳곳을 IT가 점령하고 있고, 그 속도는 계속해서 빨라지고 있습니다. 이제 IT를 모르고선 세상을 이해하기 어려운 시대가 되었습니다.\n\n이 책은 비전공자를 위한 ‘IT 기초 문법서’로, 한 번쯤 들어봤을 프로그래밍 언어, 운영체제, 네트워크, API, JSON, 데이터베이스, 이미지 처리, 프레임워크, 라이브러리, 깃(Git), 디자인 이슈 등 다양한 지식을 담고 있습니다. 복잡하게 얽혀 있어 한 번에 이해하기 어려웠던 각각의 지식들을 친숙한 이야기로 쉽게 풀어냄으로써 IT 산업의 전체적인 큰 그림을 그릴 수 있도록 도와줍니다.\n▶ 『비전공자를 위한 이해할 수 있는 IT 지식』 북트레일러\n\n* 출판사의 사정에 따라 서비스가 변경 또는 중지될 수 있습니다.\n*window7의 경우 사운드 연결이 없을시

Crawling number = 8/60


KeyboardInterrupt: 